# Explainable AI (XAI) with LIME and SHAP

**Copyright (c) 2026 Shrikara Kaudambady**

This notebook demonstrates how to use LIME (Local Interpretable Model-agnostic Explanations) and SHAP (SHapley Additive exPlanations) to interpret the predictions of a machine learning model. We will train a simple classifier on the Iris dataset and then use these XAI techniques to understand its behavior.

## 1. Setup and Data Loading

First, we import the necessary libraries and load the Iris dataset, which is a classic dataset for classification tasks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

import lime
import lime.lime_tabular
import shap

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Feature names: {iris.feature_names}")
print(f"Target names: {iris.target_names}")

## 2. Model Training

We will train a `RandomForestClassifier`. The choice of model is not critical, as both LIME and SHAP are model-agnostic.

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f"Model Accuracy: {score:.2f}")

## 3. Explainability with LIME

LIME explains a single prediction by creating a local, interpretable model (like a linear model) around it. It perturbs the input and sees how the predictions change to infer feature importance.

We will pick a single instance from the test set and see how LIME explains the model's prediction for it.

In [ ]:
# Create a LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train, 
    feature_names=iris.feature_names, 
    class_names=iris.target_names, 
    mode='classification'
)

In [ ]:
# Select an instance to explain
instance_idx = 10
instance = X_test[instance_idx]
true_label = y_test[instance_idx]

# Generate the explanation
explanation = explainer.explain_instance(
    data_row=instance, 
    predict_fn=model.predict_proba, 
    num_features=len(iris.feature_names)
)

print(f"Instance Details:")
for i, feature in enumerate(iris.feature_names):
    print(f"  {feature}: {instance[i]}")

print(f"
Prediction: {iris.target_names[model.predict(instance.reshape(1, -1))[0]]}")
print(f"True Label: {iris.target_names[true_label]}")

# Visualize the explanation
explanation.show_in_notebook(show_table=True, show_all=False)

## 4. Explainability with SHAP

SHAP (SHapley Additive exPlanations) uses a game theory approach to explain model output. It calculates Shapley values, which represent the contribution of each feature to a prediction.

SHAP can provide both global explanations (understanding the model as a whole) and local explanations (understanding a single prediction).

In [ ]:
# Initialize the SHAP explainer
shap_explainer = shap.TreeExplainer(model)

# Calculate SHAP values for the test set
shap_values = shap_explainer.shap_values(X_test)

### Global Explanations with SHAP

A SHAP summary plot gives a high-level overview of which features are most important for the model's predictions across the entire dataset.

In [ ]:
# Create a SHAP summary plot
shap.summary_plot(shap_values, X_test, class_names=iris.target_names, feature_names=iris.feature_names)

### Local Explanation with SHAP

A SHAP force plot shows how features contributed to push the model's prediction away from the base value. It's a powerful way to visualize the reasoning for a single prediction.

In [ ]:
# Initialize javascript for SHAP plots
shap.initjs()

# Explain the same instance as with LIME
instance_idx_shap = 10

# Create a force plot for one prediction
# We look at the explanation for the predicted class
predicted_class = model.predict(X_test[instance_idx_shap].reshape(1, -1))[0]

shap.force_plot(
    base_value=shap_explainer.expected_value[predicted_class],
    shap_values=shap_values[predicted_class][instance_idx_shap, :],
    features=X_test[instance_idx_shap, :],
    feature_names=iris.feature_names
)

## 5. Conclusion

This notebook provided a brief introduction to Explainable AI using LIME and SHAP.

- **LIME** is excellent for explaining individual predictions in a simple, intuitive way.
- **SHAP** provides mathematically rigorous explanations, offering both powerful global insights (summary plots) and detailed local explanations (force plots).

Incorporating these tools into the machine learning workflow is crucial for building trust, debugging models, and ensuring fairness.